## NBA statistika 2000 - 2021 regular season i playoff

Prolazi se kroz dva dataset-a gde jedan sadrzi raznu statistiku za timove izmedju 2000-2021 u regularnoj sezoni, dok drugi istu statistiku samo u playoff-u.
Izvlacimo neke interesantne informacije i neka interesantna poredjenja kako se timovi poboljsavaju ili pogorsavaju pre prelaska iz obicne lige u playoff
link ka datasetovima: https://www.kaggle.com/datasets/mharvnek/nba-team-stats-00-to-18?select=nba_team_stats_00_to_21.csv

importujemo sve sto nam treba isto kao na vezbama

In [0]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row, Column
from pyspark.sql.functions import *
from pyspark.sql.types import *

uzet je neki databricks-ov template za uzimanje dataseta, sve opcije postavljene na true da bi bilo sto preglednije, koriscen display da bi se videli svi redovi, ispod je prikazan
dataset regularne sezone (df1) a nakon njega playoff statistika (df2)

In [0]:
# File location and type
file_location = "/FileStore/tables/nba_team_stats_00_to_21.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df1 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df1)


teamstatspk,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-,SEASON
0,Phoenix Suns,52,42,10,0.808,48.1,112.7,42.7,89.4,47.8,11.5,31.7,36.3,15.8,20.0,79.1,10.2,35.9,46.1,26.5,13.3,8.6,4.3,4.0,19.3,19.3,7.8,2020-21
1,Golden State Warriors,53,40,13,0.755,48.2,110.9,40.4,86.5,46.7,14.6,40.1,36.4,15.5,20.3,76.4,10.1,36.4,46.5,27.5,15.6,9.4,4.9,4.1,20.3,17.7,8.3,2020-21
2,Memphis Grizzlies,55,37,18,0.673,48.3,112.7,42.7,93.4,45.7,11.1,32.7,33.9,16.2,22.0,73.7,13.6,35.0,48.6,25.1,13.3,10.1,6.4,6.4,19.1,19.0,4.1,2020-21
3,Miami Heat,54,34,20,0.63,48.5,108.7,39.3,85.7,45.9,13.5,36.1,37.5,16.5,20.2,81.5,10.8,33.8,44.6,25.9,14.9,7.6,3.3,4.4,20.5,20.0,4.2,2020-21
4,Chicago Bulls,53,33,20,0.623,48.1,111.6,41.6,87.0,47.8,11.2,30.0,37.2,17.2,21.2,81.4,8.9,34.1,43.0,24.5,13.0,7.2,4.6,5.2,18.8,17.8,1.7,2020-21
5,Milwaukee Bucks,55,34,21,0.618,48.2,112.7,40.7,88.9,45.8,14.3,39.4,36.4,16.9,21.6,78.2,10.3,36.5,46.8,23.4,13.7,7.7,4.2,4.5,17.8,19.2,4.0,2020-21
6,Cleveland Cavaliers,54,33,21,0.611,48.0,106.5,39.5,85.1,46.4,11.8,33.7,35.1,15.7,20.9,75.1,10.4,34.9,45.3,25.5,14.9,7.2,4.3,4.5,17.0,19.2,4.4,2020-21
7,Philadelphia 76ers,53,32,21,0.604,48.4,107.8,39.6,85.1,46.6,11.0,30.9,35.8,17.5,21.7,80.9,8.4,33.7,42.1,23.2,12.5,7.6,5.7,4.6,19.1,18.9,2.2,2020-21
8,Utah Jazz,53,32,21,0.604,48.0,113.6,40.6,85.9,47.3,14.6,40.0,36.4,17.8,22.9,77.8,10.1,35.7,45.8,22.2,14.3,7.1,4.8,4.2,18.8,20.1,6.0,2020-21
9,Dallas Mavericks,54,31,23,0.574,48.3,106.2,39.1,86.4,45.3,12.3,36.8,33.5,15.6,20.2,77.0,9.5,34.3,43.8,24.0,12.6,7.1,4.1,3.9,19.7,19.9,2.7,2020-21


In [0]:
# File location and type
file_location = "/FileStore/tables/nba_team_stats_playoffs_00_to_21.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df2 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df2)

teamstatspk,team,games_played,wins,losses,win_percentage,minutes,points,field_goals_made,field_goals_attempted,field_goal_percentage,three_pointers_made,three_pointers_attempted,three_point_percentage,free_throws_made,free_throw_attempted,free_throw_percentage,offensive_rebounds,defensive_rebounds,rebounds,assists,turnovers,steals,blocks,blocks_attempted,personal_fouls,personal_fouls_drawn,plus_minus,season
0,Milwaukee Bucks,23,16,7,0.696,48.4,110.3,42.1,91.1,46.2,11.2,35.0,32.1,14.9,20.8,71.8,12.8,36.1,49.0,22.8,12.7,7.8,4.2,3.8,18.0,20.1,5.1,2020-21
1,Phoenix Suns,22,14,8,0.636,48.0,109.0,40.9,85.0,48.1,11.2,29.7,37.6,16.0,18.6,86.1,8.0,34.7,42.7,23.2,11.9,6.5,4.2,3.6,19.7,18.5,4.5,2020-21
2,Brooklyn Nets,12,7,5,0.583,48.4,112.5,40.7,86.2,47.2,13.6,35.6,38.2,17.6,20.3,86.8,8.3,34.3,42.6,22.6,11.6,7.1,4.8,6.2,21.2,18.6,6.3,2020-21
3,Philadelphia 76ers,12,7,5,0.583,48.0,116.3,42.5,85.8,49.6,11.2,28.4,39.3,20.1,29.3,68.5,9.0,35.7,44.7,24.9,12.8,8.0,6.2,3.8,23.0,23.8,7.5,2020-21
4,Atlanta Hawks,18,10,8,0.556,48.0,106.3,38.9,86.8,44.9,11.7,34.2,34.3,16.7,20.9,79.8,9.1,33.3,42.4,20.2,12.4,6.7,4.4,4.3,21.2,20.3,-1.5,2020-21
5,Utah Jazz,11,6,5,0.545,48.0,116.9,39.7,84.2,47.2,18.0,43.6,41.3,19.5,24.3,80.1,9.4,33.0,42.4,19.6,13.5,5.2,4.1,4.3,21.5,21.1,-0.8,2020-21
6,LA Clippers,19,10,9,0.526,48.0,111.6,39.1,81.9,47.7,13.8,36.4,38.1,19.5,23.7,82.3,8.8,32.4,41.2,20.8,10.7,6.8,4.3,2.7,19.5,21.3,4.1,2020-21
7,Dallas Mavericks,7,3,4,0.429,48.0,106.0,39.1,86.0,45.5,13.6,35.0,38.8,14.1,20.0,70.7,10.3,30.3,40.6,20.7,11.0,5.6,2.1,5.0,20.3,20.6,-5.1,2020-21
8,Denver Nuggets,10,4,6,0.4,49.0,114.8,42.1,92.2,45.7,14.7,38.5,38.2,15.9,20.9,76.1,11.7,31.9,43.6,24.9,11.2,6.1,4.3,4.9,20.4,20.3,-5.5,2020-21
9,Los Angeles Lakers,6,2,4,0.333,48.0,97.5,34.3,83.0,41.4,9.8,32.8,29.9,19.0,24.5,77.6,10.5,32.7,43.2,19.3,13.8,6.7,4.0,5.0,20.5,21.0,-6.5,2020-21


sredjujemo neke kolone, izbacujemo neke nepotrebne kolone jer ih ima previse

In [0]:
#df2 = df2.withColumnRenamed("team","tim")


df2=df2.drop('teamstatspk','field_goals_made','field_goals_attempted','three_pointers_made','three_pointers_attempted',\
        'free_throws_made','free_throw_attempted','blocks_attempted','offensive_rebounds',\
             'defensive_rebounds','personal_fouls','personal_fouls_drawn')

df1=df1.drop('teamstatspk','FGM','FGA','3PM','3PA',\
        'FTM','FTA','BLKA','OREB','DREB','PF','PFD')

display(df2)
display(df1)
    

team,games_played,wins,losses,win_percentage,minutes,points,field_goal_percentage,three_point_percentage,free_throw_percentage,rebounds,assists,turnovers,steals,blocks,plus_minus,season
Milwaukee Bucks,23,16,7,0.696,48.4,110.3,46.2,32.1,71.8,49.0,22.8,12.7,7.8,4.2,5.1,2020-21
Phoenix Suns,22,14,8,0.636,48.0,109.0,48.1,37.6,86.1,42.7,23.2,11.9,6.5,4.2,4.5,2020-21
Brooklyn Nets,12,7,5,0.583,48.4,112.5,47.2,38.2,86.8,42.6,22.6,11.6,7.1,4.8,6.3,2020-21
Philadelphia 76ers,12,7,5,0.583,48.0,116.3,49.6,39.3,68.5,44.7,24.9,12.8,8.0,6.2,7.5,2020-21
Atlanta Hawks,18,10,8,0.556,48.0,106.3,44.9,34.3,79.8,42.4,20.2,12.4,6.7,4.4,-1.5,2020-21
Utah Jazz,11,6,5,0.545,48.0,116.9,47.2,41.3,80.1,42.4,19.6,13.5,5.2,4.1,-0.8,2020-21
LA Clippers,19,10,9,0.526,48.0,111.6,47.7,38.1,82.3,41.2,20.8,10.7,6.8,4.3,4.1,2020-21
Dallas Mavericks,7,3,4,0.429,48.0,106.0,45.5,38.8,70.7,40.6,20.7,11.0,5.6,2.1,-5.1,2020-21
Denver Nuggets,10,4,6,0.4,49.0,114.8,45.7,38.2,76.1,43.6,24.9,11.2,6.1,4.3,-5.5,2020-21
Los Angeles Lakers,6,2,4,0.333,48.0,97.5,41.4,29.9,77.6,43.2,19.3,13.8,6.7,4.0,-6.5,2020-21


TEAM,GP,W,L,WIN%,MIN,PTS,FG%,3P%,FT%,REB,AST,TOV,STL,BLK,+/-,SEASON
Phoenix Suns,52,42,10,0.808,48.1,112.7,47.8,36.3,79.1,46.1,26.5,13.3,8.6,4.3,7.8,2020-21
Golden State Warriors,53,40,13,0.755,48.2,110.9,46.7,36.4,76.4,46.5,27.5,15.6,9.4,4.9,8.3,2020-21
Memphis Grizzlies,55,37,18,0.673,48.3,112.7,45.7,33.9,73.7,48.6,25.1,13.3,10.1,6.4,4.1,2020-21
Miami Heat,54,34,20,0.63,48.5,108.7,45.9,37.5,81.5,44.6,25.9,14.9,7.6,3.3,4.2,2020-21
Chicago Bulls,53,33,20,0.623,48.1,111.6,47.8,37.2,81.4,43.0,24.5,13.0,7.2,4.6,1.7,2020-21
Milwaukee Bucks,55,34,21,0.618,48.2,112.7,45.8,36.4,78.2,46.8,23.4,13.7,7.7,4.2,4.0,2020-21
Cleveland Cavaliers,54,33,21,0.611,48.0,106.5,46.4,35.1,75.1,45.3,25.5,14.9,7.2,4.3,4.4,2020-21
Philadelphia 76ers,53,32,21,0.604,48.4,107.8,46.6,35.8,80.9,42.1,23.2,12.5,7.6,5.7,2.2,2020-21
Utah Jazz,53,32,21,0.604,48.0,113.6,47.3,36.4,77.8,45.8,22.2,14.3,7.1,4.8,6.0,2020-21
Dallas Mavericks,54,31,23,0.574,48.3,106.2,45.3,33.5,77.0,43.8,24.0,12.6,7.1,4.1,2.7,2020-21


Izjednacavanje imena kolona oba dataframe-a da budu uniformna

In [0]:
df1=df1.withColumnRenamed("TEAM","team").withColumnRenamed("GP","games_played").withColumnRenamed("W","wins").withColumnRenamed("L","losses")\
.withColumnRenamed("WIN%","win_percentage").withColumnRenamed("MIN","minutes").withColumnRenamed("PTS","points").withColumnRenamed("FG%","field_goal_percentage")\
.withColumnRenamed("3P%","three_point percentage").withColumnRenamed("FT%","free_throw_percentage").withColumnRenamed("REB","rebounds").withColumnRenamed("AST","assists")\
.withColumnRenamed("TOV","turnover").withColumnRenamed("STL","steals").withColumnRenamed("BLK","blocks").withColumnRenamed("+/-","plus_minus")\
.withColumnRenamed("SEASON","season")

display(df1)
display(df2)


team,games_played,wins,losses,win_percentage,minutes,points,field_goal_percentage,three_point percentage,free_throw_percentage,rebounds,assists,turnover,steals,blocks,plus_minus,season
Phoenix Suns,52,42,10,0.808,48.1,112.7,47.8,36.3,79.1,46.1,26.5,13.3,8.6,4.3,7.8,2020-21
Golden State Warriors,53,40,13,0.755,48.2,110.9,46.7,36.4,76.4,46.5,27.5,15.6,9.4,4.9,8.3,2020-21
Memphis Grizzlies,55,37,18,0.673,48.3,112.7,45.7,33.9,73.7,48.6,25.1,13.3,10.1,6.4,4.1,2020-21
Miami Heat,54,34,20,0.63,48.5,108.7,45.9,37.5,81.5,44.6,25.9,14.9,7.6,3.3,4.2,2020-21
Chicago Bulls,53,33,20,0.623,48.1,111.6,47.8,37.2,81.4,43.0,24.5,13.0,7.2,4.6,1.7,2020-21
Milwaukee Bucks,55,34,21,0.618,48.2,112.7,45.8,36.4,78.2,46.8,23.4,13.7,7.7,4.2,4.0,2020-21
Cleveland Cavaliers,54,33,21,0.611,48.0,106.5,46.4,35.1,75.1,45.3,25.5,14.9,7.2,4.3,4.4,2020-21
Philadelphia 76ers,53,32,21,0.604,48.4,107.8,46.6,35.8,80.9,42.1,23.2,12.5,7.6,5.7,2.2,2020-21
Utah Jazz,53,32,21,0.604,48.0,113.6,47.3,36.4,77.8,45.8,22.2,14.3,7.1,4.8,6.0,2020-21
Dallas Mavericks,54,31,23,0.574,48.3,106.2,45.3,33.5,77.0,43.8,24.0,12.6,7.1,4.1,2.7,2020-21


team,games_played,wins,losses,win_percentage,minutes,points,field_goal_percentage,three_point_percentage,free_throw_percentage,rebounds,assists,turnovers,steals,blocks,plus_minus,season
Milwaukee Bucks,23,16,7,0.696,48.4,110.3,46.2,32.1,71.8,49.0,22.8,12.7,7.8,4.2,5.1,2020-21
Phoenix Suns,22,14,8,0.636,48.0,109.0,48.1,37.6,86.1,42.7,23.2,11.9,6.5,4.2,4.5,2020-21
Brooklyn Nets,12,7,5,0.583,48.4,112.5,47.2,38.2,86.8,42.6,22.6,11.6,7.1,4.8,6.3,2020-21
Philadelphia 76ers,12,7,5,0.583,48.0,116.3,49.6,39.3,68.5,44.7,24.9,12.8,8.0,6.2,7.5,2020-21
Atlanta Hawks,18,10,8,0.556,48.0,106.3,44.9,34.3,79.8,42.4,20.2,12.4,6.7,4.4,-1.5,2020-21
Utah Jazz,11,6,5,0.545,48.0,116.9,47.2,41.3,80.1,42.4,19.6,13.5,5.2,4.1,-0.8,2020-21
LA Clippers,19,10,9,0.526,48.0,111.6,47.7,38.1,82.3,41.2,20.8,10.7,6.8,4.3,4.1,2020-21
Dallas Mavericks,7,3,4,0.429,48.0,106.0,45.5,38.8,70.7,40.6,20.7,11.0,5.6,2.1,-5.1,2020-21
Denver Nuggets,10,4,6,0.4,49.0,114.8,45.7,38.2,76.1,43.6,24.9,11.2,6.1,4.3,-5.5,2020-21
Los Angeles Lakers,6,2,4,0.333,48.0,97.5,41.4,29.9,77.6,43.2,19.3,13.8,6.7,4.0,-6.5,2020-21


kolona minutes je delovala bespotrebno jer je svima prosek 48 osim ako nisu igrali produzetke pa je preimenovano ako je 48min u regular a ako ima vise u overtime
da se naznaci da su igrani produzeci

In [0]:


#df2=df2.select('*',df2.minutes.cast(StringType()).alias("minuti"))
#display(df2)

df2 = df2.withColumn('minutes',when(df2.minutes == 48.0 ,regexp_replace('minutes','48','Regular')).otherwise('Overtime'))
df1 = df1.withColumn('minutes',when(df1.minutes == 48.0 ,regexp_replace('minutes','48','Regular')).otherwise('Overtime'))
#df2 = df2.withColumn("minutes",col("minutes").cast("String"))
display(df1)
display(df2)


team,games_played,wins,losses,win_percentage,minutes,points,field_goal_percentage,three_point percentage,free_throw_percentage,rebounds,assists,turnover,steals,blocks,plus_minus,season
Phoenix Suns,52,42,10,0.808,Overtime,112.7,47.8,36.3,79.1,46.1,26.5,13.3,8.6,4.3,7.8,2020-21
Golden State Warriors,53,40,13,0.755,Overtime,110.9,46.7,36.4,76.4,46.5,27.5,15.6,9.4,4.9,8.3,2020-21
Memphis Grizzlies,55,37,18,0.673,Overtime,112.7,45.7,33.9,73.7,48.6,25.1,13.3,10.1,6.4,4.1,2020-21
Miami Heat,54,34,20,0.63,Overtime,108.7,45.9,37.5,81.5,44.6,25.9,14.9,7.6,3.3,4.2,2020-21
Chicago Bulls,53,33,20,0.623,Overtime,111.6,47.8,37.2,81.4,43.0,24.5,13.0,7.2,4.6,1.7,2020-21
Milwaukee Bucks,55,34,21,0.618,Overtime,112.7,45.8,36.4,78.2,46.8,23.4,13.7,7.7,4.2,4.0,2020-21
Cleveland Cavaliers,54,33,21,0.611,Regular.0,106.5,46.4,35.1,75.1,45.3,25.5,14.9,7.2,4.3,4.4,2020-21
Philadelphia 76ers,53,32,21,0.604,Overtime,107.8,46.6,35.8,80.9,42.1,23.2,12.5,7.6,5.7,2.2,2020-21
Utah Jazz,53,32,21,0.604,Regular.0,113.6,47.3,36.4,77.8,45.8,22.2,14.3,7.1,4.8,6.0,2020-21
Dallas Mavericks,54,31,23,0.574,Overtime,106.2,45.3,33.5,77.0,43.8,24.0,12.6,7.1,4.1,2.7,2020-21


team,games_played,wins,losses,win_percentage,minutes,points,field_goal_percentage,three_point_percentage,free_throw_percentage,rebounds,assists,turnovers,steals,blocks,plus_minus,season
Milwaukee Bucks,23,16,7,0.696,Overtime,110.3,46.2,32.1,71.8,49.0,22.8,12.7,7.8,4.2,5.1,2020-21
Phoenix Suns,22,14,8,0.636,Regular.0,109.0,48.1,37.6,86.1,42.7,23.2,11.9,6.5,4.2,4.5,2020-21
Brooklyn Nets,12,7,5,0.583,Overtime,112.5,47.2,38.2,86.8,42.6,22.6,11.6,7.1,4.8,6.3,2020-21
Philadelphia 76ers,12,7,5,0.583,Regular.0,116.3,49.6,39.3,68.5,44.7,24.9,12.8,8.0,6.2,7.5,2020-21
Atlanta Hawks,18,10,8,0.556,Regular.0,106.3,44.9,34.3,79.8,42.4,20.2,12.4,6.7,4.4,-1.5,2020-21
Utah Jazz,11,6,5,0.545,Regular.0,116.9,47.2,41.3,80.1,42.4,19.6,13.5,5.2,4.1,-0.8,2020-21
LA Clippers,19,10,9,0.526,Regular.0,111.6,47.7,38.1,82.3,41.2,20.8,10.7,6.8,4.3,4.1,2020-21
Dallas Mavericks,7,3,4,0.429,Regular.0,106.0,45.5,38.8,70.7,40.6,20.7,11.0,5.6,2.1,-5.1,2020-21
Denver Nuggets,10,4,6,0.4,Overtime,114.8,45.7,38.2,76.1,43.6,24.9,11.2,6.1,4.3,-5.5,2020-21
Los Angeles Lakers,6,2,4,0.333,Regular.0,97.5,41.4,29.9,77.6,43.2,19.3,13.8,6.7,4.0,-6.5,2020-21


Filtriramo redove koji su vezani za 2020-21 sezonu jer za njih nemamo statistiku za sve odigrane utakmice (verovatno je tada pravljen dataset)

In [0]:
df1 = df1.filter(df1.season != "2020-21")
df2 = df2.filter(df2.season != "2020-21")



display(df1)
display(df2)

team,games_played,wins,losses,win_percentage,minutes,points,field_goal_percentage,three_point percentage,free_throw_percentage,rebounds,assists,turnover,steals,blocks,plus_minus,season
Milwaukee Bucks,73,56,17,0.8,Overtime,118.7,47.6,35.5,74.2,51.7,25.9,15.1,7.2,5.9,10.1,2019-20
Toronto Raptors,72,53,19,0.7,Overtime,112.8,45.8,37.4,79.6,45.4,25.2,14.8,8.8,5.0,6.2,2019-20
Los Angeles Lakers,71,52,19,0.7,Overtime,113.4,48.0,34.9,72.9,45.7,25.4,15.2,8.6,6.6,5.8,2019-20
LA Clippers,72,49,23,0.7,Overtime,116.3,46.6,37.1,79.1,47.7,23.7,14.6,7.1,4.7,6.4,2019-20
Boston Celtics,72,48,24,0.7,Overtime,113.7,46.1,36.4,80.1,46.1,23.0,13.8,8.3,5.6,6.3,2019-20
Denver Nuggets,73,46,27,0.6,Overtime,111.3,47.3,35.9,77.7,44.1,26.7,13.8,8.0,4.6,2.1,2019-20
Indiana Pacers,73,45,28,0.6,Overtime,109.4,47.6,36.3,78.7,42.8,25.9,13.2,7.4,5.2,2.0,2019-20
Houston Rockets,72,44,28,0.6,Overtime,117.8,45.1,34.5,79.1,44.3,21.6,14.7,8.7,5.2,3.0,2019-20
Oklahoma City Thunder,72,44,28,0.6,Overtime,110.4,46.8,35.5,79.6,42.9,21.7,13.7,7.6,4.9,2.0,2019-20
Utah Jazz,72,44,28,0.6,Overtime,111.3,47.1,38.0,77.9,44.9,22.4,15.1,6.1,4.1,2.5,2019-20


team,games_played,wins,losses,win_percentage,minutes,points,field_goal_percentage,three_point_percentage,free_throw_percentage,rebounds,assists,turnovers,steals,blocks,plus_minus,season
Los Angeles Lakers,21,16,5,0.8,Regular.0,112.8,48.9,35.4,75.8,44.1,25.6,15.4,8.0,5.3,6.9,2019-20
Miami Heat,21,14,7,0.7,Overtime,110.0,46.0,35.9,82.5,41.3,24.9,13.2,6.9,3.9,2.0,2019-20
Toronto Raptors,11,7,4,0.6,Overtime,109.4,44.2,36.3,77.8,44.6,24.8,13.1,7.1,5.5,4.1,2019-20
Boston Celtics,17,10,7,0.6,Overtime,109.4,45.4,34.1,81.6,45.2,22.6,13.9,6.7,4.8,5.0,2019-20
LA Clippers,13,7,6,0.5,Overtime,114.0,47.4,36.8,79.8,44.0,21.2,13.0,7.5,3.7,4.2,2019-20
Milwaukee Bucks,10,5,5,0.5,Overtime,111.1,46.0,35.8,72.4,46.7,24.3,14.9,7.2,4.2,1.2,2019-20
Denver Nuggets,19,9,10,0.5,Overtime,107.4,47.1,37.9,81.6,40.6,22.5,13.9,6.2,4.0,-2.3,2019-20
Oklahoma City Thunder,7,3,4,0.4,Overtime,104.0,43.0,32.8,77.4,49.9,17.9,17.0,5.3,4.0,-6.4,2019-20
Utah Jazz,7,3,4,0.4,Overtime,113.4,48.7,42.1,77.3,41.1,21.4,12.0,6.1,3.1,3.4,2019-20
Houston Rockets,12,5,7,0.4,Overtime,107.7,43.6,36.2,78.9,38.8,21.3,13.4,9.1,3.5,0.8,2019-20


Zaokruzujemo win_percentage na jednu decimalu da bude bolji racun i da bude konzistentno sa ostalim percentage-ima (svi ostali su na jednoj decimali)

In [0]:
df1 = df1.withColumn('win_percentage',round(df1.win_percentage,1))
display(df1)

df2 = df2.withColumn('win_percentage',round(df2.win_percentage,1))
display(df2)

team,games_played,wins,losses,win_percentage,minutes,points,field_goal_percentage,three_point percentage,free_throw_percentage,rebounds,assists,turnover,steals,blocks,plus_minus,season
Milwaukee Bucks,73,56,17,0.8,Overtime,118.7,47.6,35.5,74.2,51.7,25.9,15.1,7.2,5.9,10.1,2019-20
Toronto Raptors,72,53,19,0.7,Overtime,112.8,45.8,37.4,79.6,45.4,25.2,14.8,8.8,5.0,6.2,2019-20
Los Angeles Lakers,71,52,19,0.7,Overtime,113.4,48.0,34.9,72.9,45.7,25.4,15.2,8.6,6.6,5.8,2019-20
LA Clippers,72,49,23,0.7,Overtime,116.3,46.6,37.1,79.1,47.7,23.7,14.6,7.1,4.7,6.4,2019-20
Boston Celtics,72,48,24,0.7,Overtime,113.7,46.1,36.4,80.1,46.1,23.0,13.8,8.3,5.6,6.3,2019-20
Denver Nuggets,73,46,27,0.6,Overtime,111.3,47.3,35.9,77.7,44.1,26.7,13.8,8.0,4.6,2.1,2019-20
Indiana Pacers,73,45,28,0.6,Overtime,109.4,47.6,36.3,78.7,42.8,25.9,13.2,7.4,5.2,2.0,2019-20
Houston Rockets,72,44,28,0.6,Overtime,117.8,45.1,34.5,79.1,44.3,21.6,14.7,8.7,5.2,3.0,2019-20
Oklahoma City Thunder,72,44,28,0.6,Overtime,110.4,46.8,35.5,79.6,42.9,21.7,13.7,7.6,4.9,2.0,2019-20
Utah Jazz,72,44,28,0.6,Overtime,111.3,47.1,38.0,77.9,44.9,22.4,15.1,6.1,4.1,2.5,2019-20


team,games_played,wins,losses,win_percentage,minutes,points,field_goal_percentage,three_point_percentage,free_throw_percentage,rebounds,assists,turnovers,steals,blocks,plus_minus,season
Los Angeles Lakers,21,16,5,0.8,Regular.0,112.8,48.9,35.4,75.8,44.1,25.6,15.4,8.0,5.3,6.9,2019-20
Miami Heat,21,14,7,0.7,Overtime,110.0,46.0,35.9,82.5,41.3,24.9,13.2,6.9,3.9,2.0,2019-20
Toronto Raptors,11,7,4,0.6,Overtime,109.4,44.2,36.3,77.8,44.6,24.8,13.1,7.1,5.5,4.1,2019-20
Boston Celtics,17,10,7,0.6,Overtime,109.4,45.4,34.1,81.6,45.2,22.6,13.9,6.7,4.8,5.0,2019-20
LA Clippers,13,7,6,0.5,Overtime,114.0,47.4,36.8,79.8,44.0,21.2,13.0,7.5,3.7,4.2,2019-20
Milwaukee Bucks,10,5,5,0.5,Overtime,111.1,46.0,35.8,72.4,46.7,24.3,14.9,7.2,4.2,1.2,2019-20
Denver Nuggets,19,9,10,0.5,Overtime,107.4,47.1,37.9,81.6,40.6,22.5,13.9,6.2,4.0,-2.3,2019-20
Oklahoma City Thunder,7,3,4,0.4,Overtime,104.0,43.0,32.8,77.4,49.9,17.9,17.0,5.3,4.0,-6.4,2019-20
Utah Jazz,7,3,4,0.4,Overtime,113.4,48.7,42.1,77.3,41.1,21.4,12.0,6.1,3.1,3.4,2019-20
Houston Rockets,12,5,7,0.4,Overtime,107.7,43.6,36.2,78.9,38.8,21.3,13.4,9.1,3.5,0.8,2019-20


Koristimo window partition da grupisemo timove i sortiramo ih po pobedima koliko su imali u sezonama

In [0]:
from pyspark.sql.window import Window
 
windowSpec  = Window.partitionBy("team").orderBy("wins")

df1 = df1.withColumn("rank_po_sezonama",row_number().over(windowSpec))
display(df1)
#dfSorted = dfSorted.select("teamstatspk","W")

#poffWinners = df2.select("*").where(df2.wins==16).orderBy('season', ascending = False)
df2 = df2.withColumn("rank_po_sezonama",row_number().over(windowSpec))
display(df2)

#dfJoined = poffWinners.join(dfSorted, poffWinners.teamstatspk == dfSorted.teamstatspk,)
#display(dfJoined)

team,games_played,wins,losses,win_percentage,minutes,points,field_goal_percentage,three_point percentage,free_throw_percentage,rebounds,assists,turnover,steals,blocks,plus_minus,season,rank_po_sezonama
Atlanta Hawks,82,13,69,0.2,Overtime,92.7,44.1,31.2,71.1,41.9,19.7,16.1,7.7,4.2,-9.7,2004-05,1
Atlanta Hawks,67,20,47,0.3,Overtime,111.8,44.9,33.3,79.0,43.3,24.0,16.2,7.8,5.1,-8.0,2019-20,2
Atlanta Hawks,82,24,58,0.3,Overtime,103.4,44.6,36.0,78.5,41.9,23.7,15.5,7.8,4.2,-5.5,2017-18,3
Atlanta Hawks,82,25,57,0.3,Overtime,91.0,43.1,35.7,75.9,42.9,19.0,16.7,7.7,4.7,-5.2,2000-01,4
Atlanta Hawks,82,26,56,0.3,Overtime,97.2,45.4,36.7,75.0,40.3,19.8,15.7,7.2,4.8,-4.8,2005-06,5
Atlanta Hawks,82,28,54,0.3,Overtime,92.8,43.3,33.5,77.6,42.7,20.1,16.5,7.6,5.0,-4.6,2003-04,6
Atlanta Hawks,82,29,53,0.4,Overtime,113.3,45.1,35.2,75.2,46.1,25.8,17.0,8.2,5.1,-6.0,2018-19,7
Atlanta Hawks,82,30,52,0.4,Overtime,93.7,44.4,32.9,76.1,40.1,19.2,15.9,7.4,5.4,-4.8,2006-07,8
Atlanta Hawks,82,33,49,0.4,Overtime,94.0,43.9,35.4,76.5,41.5,20.2,15.5,8.1,4.3,-4.2,2001-02,9
Atlanta Hawks,82,35,47,0.4,Overtime,94.1,44.4,35.2,79.3,42.6,20.5,16.7,7.5,5.8,-3.6,2002-03,10


team,games_played,wins,losses,win_percentage,minutes,points,field_goal_percentage,three_point_percentage,free_throw_percentage,rebounds,assists,turnovers,steals,blocks,plus_minus,season,rank_po_sezonama
Atlanta Hawks,6,2,4,0.3,Regular.0,105.5,44.4,30.5,76.4,44.3,22.3,16.0,6.0,3.3,-1.2,2016-17,1
Atlanta Hawks,6,2,4,0.3,Regular.0,89.3,42.2,33.0,68.2,38.3,19.2,12.2,7.3,3.8,-5.2,2012-13,2
Atlanta Hawks,6,2,4,0.3,Overtime,82.2,40.1,31.0,78.4,42.5,17.5,15.3,5.5,3.3,-4.7,2011-12,3
Atlanta Hawks,7,3,4,0.4,Regular.0,92.4,38.5,34.3,82.8,39.3,16.9,13.6,6.6,4.6,-1.0,2013-14,4
Atlanta Hawks,7,3,4,0.4,Regular.0,87.1,41.1,32.2,78.2,38.3,17.4,15.9,4.3,5.7,-12.0,2007-08,5
Atlanta Hawks,10,4,6,0.4,Overtime,100.1,43.2,33.3,79.5,44.3,22.6,14.4,7.5,7.6,-1.0,2015-16,6
Atlanta Hawks,11,4,7,0.4,Regular.0,89.5,42.5,33.3,78.5,39.9,16.4,11.7,6.4,5.1,-6.8,2009-10,7
Atlanta Hawks,11,4,7,0.4,Regular.0,84.0,40.9,34.7,74.4,34.9,16.9,12.1,6.9,3.4,-7.1,2008-09,8
Atlanta Hawks,12,6,6,0.5,Regular.0,86.3,43.4,32.3,72.6,37.2,18.1,12.4,6.3,5.0,-4.4,2010-11,9
Atlanta Hawks,16,8,8,0.5,Overtime,98.0,43.1,31.7,78.1,43.7,23.8,12.7,8.5,4.9,-0.2,2014-15,10


Uploadovanje na dbfs

In [0]:
df1.write.option("header",True).option("delimiter",",").csv("dbfs:/FileStore/tables/transformed_nbaReg.csv")
df2.write.option("header",True).option("delimiter",",").csv("dbfs:/FileStore/tables/transformed_nbaPoff.csv")